In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

from pathlib import Path

from utils.helper import find_project_root

In [ ]:
DATASET_PATH = find_project_root() / Path("datasets/nlp/")

df = pd.read_csv(DATASET_PATH / "IMDB Dataset.csv.zip")

In [ ]:
df.head()

In [ ]:
from nlp.preprocessing import TextPreprocessor

preprocessor = TextPreprocessor()

sample = "This movie was NOT good!!! 😡"
print(preprocessor.preprocess(sample))
